In [0]:
from pyspark.sql.functions import current_timestamp, col, dayofmonth, date_format
from pyspark.sql.types import TimestampType

In [0]:
json_file_path = "dbfs:/FileStore/tables/GHarchive/"

In [0]:
df = spark.read.format('json').load(json_file_path)

In [0]:
df = df.withColumn('created_at', col('created_at').cast(TimestampType()))

In [0]:
df = df.filter((dayofmonth(col('created_at')) >= 16))

In [0]:
spark.conf.set('spark.sql.session.timeZone', 'America/New_York')

In [0]:
df = df.withColumn("ingested_time", current_timestamp())

In [0]:
spark.conf.set('spark.sql.session.timeZone', 'UTC')

In [0]:
%run './ADLS Setup Variables_SP'

In [0]:
(df
 .withColumn("created_day", date_format("created_at", "yyyy-MM-dd"))
 .repartition("created_day")
 .write
 .format("parquet")
 .option("header", True)
 .option("parquet.block.size", 128 * 1024 * 1024)
 .option("maxRecordsPerFile", 160000) 
 .partitionBy("created_day")
 .mode("overwrite").save(bronze_path2)
)